# Trabalho Prático de Aprendizado Descritivo
Gabriel Bifano Freddi

João Vitor Santana Depollo

Pedro de Oliveira Guedes

Tarcízio Augusto Santos Lafaiete

Vinícius Alves de Faria Resende


## Obtenção dos dados
Para fazer a obtenção dos dados será utilizado o Google Drive, já que para fins de colaboração em tempo real, está sendo utilizada a plataforma Google Colab para desenvolvimento do script.

A célula abaixo faz a integração do ambiente de desenvolvimento com a conta do Google Drive do usuário, ela só precisa ser executada uma vez por sessão.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Os dados foram originalmente coletados por meio de _web scrapping_, utilizando a API pública da Steam para obter dados no intervalo de 23/04/2022 até 04/07/2023 de todos os jogos existentes na plataforma. Para baixar os arquivos, se refira ao [link do Kaggle](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data?select=games.csv). O autor da publicação disponibiliza também um link para o [script Python do _web scrapper_](https://github.com/FronkonGames/Steam-Games-Scraper) utilizado.

Com os arquivos baixados em uma compressão `.zip`, descomprima-os e envie para o caminho `MyDrive/steam_database` para que possa ser acessado no código corretamente.

In [ ]:
import pandas as pd

games_df = pd.read_csv('/content/drive/MyDrive/steam_database/games.csv')

games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85103 entries, 0 to 85102
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85103 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85103 non-null  object 
 3   Estimated owners            85103 non-null  object 
 4   Peak CCU                    85103 non-null  int64  
 5   Required age                85103 non-null  int64  
 6   Price                       85103 non-null  float64
 7   DLC count                   85103 non-null  int64  
 8   About the game              81536 non-null  object 
 9   Supported languages         85103 non-null  object 
 10  Full audio languages        85103 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85103 non-null  object 
 13  Website                     394

## Exploração dos dados
Inicialmente, é necessário visualizar os dados do dataset e fazer observações sobre a necessidade das colunas.

Já ao observar as informações da célula anterior sobre o _dataset_, é possível verificar que a quantidade de valores não nulos da coluna `Name` difere do total. A ausência do nome de um jogo impede que percepções úteis sejam feitas sobre ele, dessa forma devem ser eliminadas as linhas com valores nulos nessa coluna.

In [ ]:
games_with_name_df = games_df.dropna(subset=['Name'])
games_with_name_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85097 entries, 0 to 85102
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85097 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85097 non-null  object 
 3   Estimated owners            85097 non-null  object 
 4   Peak CCU                    85097 non-null  int64  
 5   Required age                85097 non-null  int64  
 6   Price                       85097 non-null  float64
 7   DLC count                   85097 non-null  int64  
 8   About the game              81535 non-null  object 
 9   Supported languages         85097 non-null  object 
 10  Full audio languages        85097 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85097 non-null  object 
 13  Website                     39452 no

Para trazer informações mais úteis sobre a tabela e de forma estruturada, é possível definir uma função que descreva o _dataset_ com as variáveis desejadas.

Além da melhor visualização das informações, isso também pode ser benéfico para filtrar dados pelas métricas observadas de forma facilitada.

In [ ]:
def describe_data(df: pd.DataFrame) -> pd.DataFrame:
    data_description = {
        'non_null_amount': {},
        'null_amount': {},
        'unique_values': {},
        'data_type': {},
    }
    for column in df.columns:
        data_description['non_null_amount'][column] = df[column].notnull().sum()
        data_description['null_amount'][column] = df[column].isnull().sum()
        data_description['unique_values'][column] = df[column].nunique()
        data_description['data_type'][column] = df[column].dtype

    return pd.DataFrame(data_description)

A utilização da função anterior pode auxiliar na identificação, e possível descarte, de colunas que tenham muitos valores nulos.

In [ ]:
games_df_description = describe_data(games_with_name_df)
games_df_description

,non_null_amount,null_amount,unique_values,data_type
AppID,85097,0,85097,int64
Name,85097,0,84367,object
Release date,85097,0,4469,object
Estimated owners,85097,0,14,object
Peak CCU,85097,0,1445,int64
Required age,85097,0,19,int64
Price,85097,0,584,float64
DLC count,85097,0,95,int64
About the game,81535,3562,81099,object
Supported languages,85097,0,11305,object


Com as informações obtidas pela função de descrição dos dados, várias operações podem ser feitas para tornar o dataset mais apropriado para análises.

### Deduplicação de dados
Perceba que a quantidade de nomes de jogos únicos é diferente da quantidade de jogos totais, indicando que alguns dados foram processados incorretamente.

In [ ]:
games_with_name_df['Name'].value_counts().head(10)

,count
Name,
Shadow of the Tomb Raider: Definitive Edition,20
Alone,6
Escape,5
Tom Clancy's Rainbow Six® Siege,5
Aurora,5
Train Simulator Classic,5
Jewel Quest Pack,4
Bounce,4
Lost,4


É possível apenas remover as duplicatas do _dataset_ pelo nome, mas isso poderia levar a remover uma das duplicatas menos úteis, ou que menos representam o jogo listado. Para verificar essa possibilidade, será feita a visualização de colunas importantes para as 5 cópias de `Tom Clancy's Rainbow Six® Siege`.

In [ ]:
games_with_name_df[games_with_name_df['Name'] == 'Tom Clancy\'s Rainbow Six® Siege'][['Name', 'Price', 'Estimated owners', 'Genres', 'Positive', 'Negative']]

,Name,Price,Estimated owners,Genres,Positive,Negative
2904,Tom Clancy's Rainbow Six® Siege,19.99,0 - 20000,Action,312232,64137
4287,Tom Clancy's Rainbow Six® Siege,19.99,0 - 20000,Action,312816,64201
8256,Tom Clancy's Rainbow Six® Siege,19.99,20000000 - 50000000,Action,929372,138530
21190,Tom Clancy's Rainbow Six® Siege,19.99,0 - 20000,Action,312719,64188
38967,Tom Clancy's Rainbow Six® Siege,19.99,0 - 20000,Action,312397,64151


Perceba que a maioria dos dados está duplicada, sendo que as colunas com grande diferença nos dados é `Estimated owners` e as de avaliação (`Positive` e `Negative`). Enquanto os valores das últimas duas mencionadas estão livres, é possível verificar que a estimativa de donos do jogo está classificada em intervalos.

In [ ]:
games_with_name_df['Estimated owners'].value_counts()

,count
Estimated owners,
0 - 20000,55284
0 - 0,11499
20000 - 50000,7808
50000 - 100000,3886
100000 - 200000,2566
200000 - 500000,2142
500000 - 1000000,906
1000000 - 2000000,521
2000000 - 5000000,329


Como o objetivo é manter a coluna que represente melhor os dados sobre o jogo, é possível fazer a remoção das duplicatas levando em consideração a ordem de `Estimated owners` e manter a ocorrencia com o maior valor.

Para isso, será calculada a média dos intervalos e o valor resultante dará origem a uma nova coluna `Estimated owners mean`.

In [ ]:
owners_ranges = [range.split(' - ') for range in games_with_name_df['Estimated owners']]
owners_ranges = [(int(range[0]), int(range[1])) for range in owners_ranges]

games_with_name_df['Estimated owners mean'] = [((owner_range[0] + owner_range[1]) // 2) for owner_range in owners_ranges]

games_with_name_df['Estimated owners mean'].value_counts()


<ipython-input-313-fde90922f6ad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_with_name_df['Estimated owners mean'] = [((owner_range[0] + owner_range[1]) // 2) for owner_range in owners_ranges]


,count
Estimated owners mean,
10000,55284
0,11499
35000,7808
75000,3886
150000,2566
350000,2142
750000,906
1500000,521
3500000,329


Com o processamento anterior aplicado, é possível ordenar as linhas pelo novo valor de `Estimated owners` e, ao fazer a remoção de duplicados, escolher o primeiro ou o último valor encontrado para permanecer.

De forma mais específica, o _dataset_ será ordenado de forma decrescente com base na coluna e será escolhida a primeira ocorrência do nome duplicado.

In [ ]:
games_with_name_df = games_with_name_df.sort_values(by='Estimated owners mean', ascending=False)

games_with_name_df = games_with_name_df.drop_duplicates(subset=['Name'], keep='first')
games_with_name_df[games_with_name_df['Name'] == 'Tom Clancy\'s Rainbow Six® Siege'][['Name', 'Price', 'Estimated owners', 'Genres', 'Positive', 'Negative']]

,Name,Price,Estimated owners,Genres,Positive,Negative
8256,Tom Clancy's Rainbow Six® Siege,19.99,20000000 - 50000000,Action,929372,138530


Com isso a deduplicação está finalizada.

### Tratamento de nulos
Como visualmente são poucas colunas que possuem valores nulos, é possível verificá-las de forma direta e avaliar como elas devem ser tratadas.

In [ ]:
games_df_description[games_df_description['null_amount'] > 0]

,non_null_amount,null_amount,unique_values,data_type
About the game,81535,3562,81099,object
Reviews,9743,75354,9646,object
Website,39452,45645,32199,object
Support url,41586,43511,27297,object
Support email,71507,13590,42081,object
Metacritic url,3912,81185,3814,object
Score rank,44,85053,4,float64
Notes,13018,72079,10567,object
Developers,81516,3581,49870,object
Publishers,81236,3861,43366,object


A primeira vista, é possível ver que a coluna `Score rank` possui uma quantidade muito alta de valores nulos, não justificando o esforço de tratativas para mantê-la.

In [ ]:
games_with_name_df = games_with_name_df.drop(columns=['Score rank'])

O comportamento de grandes quantidades de nulos também se repete, ainda que em menor proporção, para as colunas `Reviews`, `Metacritic url` e `Notes`. Porém, por terem maior quantidade de dados, é importante que sejam vistos alguns exemplos de valores dessas colunas antes de tomar qualquer decisão.

In [ ]:
def get_sample_data(df: pd.DataFrame, columns: list, sample_size: int) -> pd.DataFrame:
    samples = {}
    for column in columns:
        samples[column] = [sample for sample in df[df[column].notnull()][column].sample(sample_size)]
    return pd.DataFrame(samples)

samples = get_sample_data(games_with_name_df, ['Reviews', 'Metacritic url', 'Notes'], 10)
samples

,Reviews,Metacritic url,Notes
0,“It goes without saying that the comics are lo...,http://www.metacritic.com/game/pc/arma-iii?fta...,Liz ~The Tower and the Grimoire~ contains stro...
1,“A significant step forward for the king of mi...,https://www.metacritic.com/game/pc/bee-simulat...,"There are cartoony depicitions of murders, ass..."
2,“DOOR gently impressed me with its unique aest...,https://www.metacritic.com/game/pc/apex-constr...,Contains genitalia censored sexual intercourse...
3,“If you were afraid that the idea of a fighter...,https://www.metacritic.com/game/pc/rising-stor...,This Game may contain content not appropriate ...
4,"“A new, amazing hardcore 2D game for lovers of...",http://www.metacritic.com/game/pc/star-wars-ba...,All characters interact sexually. Most of thes...
5,"“It has a bunch of strange and clever puzzles,...",https://www.metacritic.com/game/pc/grime?ftag=...,Contains frequent violence and blood.
6,“I definitely wouldn't want to buy a chicken t...,https://www.metacritic.com/game/pc/roki?ftag=M...,This game is intended only for mature audience...
7,“It's an adventure that's just really delightf...,https://www.metacritic.com/game/pc/everyday-sh...,The game graphically depicts sex in still imag...
8,“Little Inner Monsters is a simple card game t...,https://www.metacritic.com/game/pc/expeditions...,"Frequent Violence or Gore, General Mature Cont..."
9,"“I'm having a lot of fun with the game, settin...",https://www.metacritic.com/game/pc/grand-theft...,"Cartoon Violence, Mild Blood depiction"


Perceba que as linhas não estão correlacionadas entre si, sendo apenas exemplos retirados aleatoriamente do conjunto de dados para visualização do formato dos valores.

Note também que, embora seja possível fazer uma análise de sentimentos dos textos na coluna `Reviews`, os resultados provavelmente seriam redundantes em relação às métricas de avaliação do jogo. Além disso, é possível verificar que ocorre a presença de textos em linguas diferentes do inglês nos dados, o que tornaria a tarefa ainda mais custosa e imprecisa de ser feita. Ao invés disso, uma alternativa viável pode ser a binarização da coluna, onde os jogos assumirão valor `True` quando uma review tiver sido feita e `False` no caso contrário.

A coluna `Metacritic url` por outro lado, por se tratar de URLs, não há um aproveitamento fácil de ser feito dos dados, portanto, será descartada. A coluna `Notes`, por outro lado, poderia ser bem utilizada no caso de dados estruturados com rótulos pré-definidos, mas o preenchimento parece ter sido deixado a critério do autor do jogo, o que dificulta a utilização da mesma. Sendo assim, ambas colunas serão removidas.

In [ ]:
games_with_name_df['has_review'] = games_with_name_df['Reviews'].notnull()

games_with_name_df = games_with_name_df.drop(columns=['Metacritic url','Notes', 'Reviews'])

games_df_description = describe_data(games_with_name_df)
games_df_description[games_df_description['null_amount'] > 0].sort_values(by='null_amount', ascending=False)

,non_null_amount,null_amount,unique_values,data_type
Website,39148,45219,32044,object
Support url,41268,43099,27138,object
Tags,63506,20861,56707,object
Support email,70933,13434,41759,object
Movies,77988,6379,77982,object
Categories,79790,4577,5633,object
Publishers,80527,3840,43037,object
Developers,80802,3565,49516,object
About the game,80819,3548,80498,object
Genres,80826,3541,2460,object


Os atributos remanescentes com quantidades valores nulos que se destacam em relação aos demais são: `Website`, `Support url`, `Support email`. Enquanto `Tags` também possui muitos valores nulos, essa coluna será tratada posteriormente em tempo oportuno.

Serão verificados exemplos de valores de cada um desses atributos para uma tomada de decisão bem informada quanto a eles.

In [ ]:
samples = get_sample_data(games_with_name_df, ['Website', 'Support url', 'Support email'], 10)
samples

,Website,Support url,Support email
0,https://www.detune.co.jp/KORG_Gadget_VR.html,https://www.digipen.edu/,potionc@qq.com
1,https://halfgeekstudios.wordpress.com/star-nom...,https://www.facebook.com/DnovelGames/,jperalta@flynnsarcades.com
2,https://www.thislandmyland.com/,https://weibo.com/fangshengmeng,support@darkinertia.com
3,http://narvalous.org/clash-of-magic/,http://www.polayart.co.kr,2143224134@qq.com
4,http://www.humongous.com,http://absolutist.com/support.html,eyaura@yahoo.com
5,http://livescreamgame.com,https://twitter.com/QQQQQQQ_______,support@afterthoughtgames.com
6,https://www.kemco-games.com/,https://support.square-enix-games.com/,support@keeptalkinggame.com
7,https://kurosawacreate.jp/bias,https://www.spacetronaut.co,feverdreamgameworks@gmail.com
8,http://www.project-ynp.com/product/cf4/,http://www.arenatacticsgame.com/,planta.developer@gmail.com
9,http://www.an-alien-with-a-magnet.com,http://sodadungeon.com,shadowmaxfs@163.com


Os atributos evidenciados não possuem formatos particularmente úteis para que análises textuais sejam feitas, portanto, podem ser eliminados ou transformados em colunas binárias.

Buscando manter o máximo de informações originais no dataset, será optado pela binarização desses valores.

In [ ]:
games_with_name_df['has_website'] = games_with_name_df['Website'].notnull()
games_with_name_df['has_support_url'] = games_with_name_df['Support url'].notnull()
games_with_name_df['has_support_email'] = games_with_name_df['Support email'].notnull()

games_with_name_df = games_with_name_df.drop(columns=['Website', 'Support url', 'Support email'])

games_df_description = describe_data(games_with_name_df)
games_df_description[games_df_description['null_amount'] > 0].sort_values(by='null_amount', ascending=False)

,non_null_amount,null_amount,unique_values,data_type
Tags,63506,20861,56707,object
Movies,77988,6379,77982,object
Categories,79790,4577,5633,object
Publishers,80527,3840,43037,object
Developers,80802,3565,49516,object
About the game,80819,3548,80498,object
Genres,80826,3541,2460,object
Screenshots,82375,1992,82375,object


As colunas `Movies`, `About the game` e `Screenshots`, a princípio, não parecem ser de ajuda para a execução da tarefa de descoberta de subgrupos. Para ter certeza, será feita a visualização de exemplos dos dados.

In [ ]:
samples = get_sample_data(games_with_name_df, ['Movies', 'About the game', 'Screenshots'], 1)
samples

,Movies,About the game,Screenshots
0,http://cdn.akamai.steamstatic.com/steam/apps/2...,This is a word game. The whole game is mainly ...,https://cdn.akamai.steamstatic.com/steam/apps/...


Como a descrição do jogo, presente na coluna `About the game` tende a não apresentar conteúdos que não sejam positivos, já que os desenvolvedores costumam descrever o jogo de forma que as pessoas queiram de fato jogá-lo, foi optado por não realizar atividades de processamento textual nela e apenas descartá-la.

Ao observar as colunas `Screeshots` e `Movies`, é possível verificar que elas são na verdade listas de urls. Sendo assim, é possível criar uma nova coluna com a contagem de URLs de cada jogo, onde um jogo que não possua _screenshots_ ou _movies_ terá contagem igual a zero.

In [ ]:
# Removendo coluna "About the game"
games_with_name_df = games_with_name_df.drop(columns=['About the game'])

In [ ]:
# Contagem de valores na coluna "Screenshots"
games_with_name_df['screenshots_amount'] = [len(screenshots.split(',')) for screenshots in games_with_name_df['Screenshots'].fillna('')]

games_with_name_df = games_with_name_df.drop(columns=['Screenshots'])

games_with_name_df['screenshots_amount'].value_counts().head(10)

,count
screenshots_amount,
5,21890
6,11842
7,8612
8,7839
10,6037
9,5769
11,3437
12,3148
1,2754


In [ ]:
# Contagem de valores na coluna "Movies"
games_with_name_df['movies_amount'] = [len(movies.split(',')) for movies in games_with_name_df['Movies'].fillna('')]

games_with_name_df = games_with_name_df.drop(columns=['Movies'])

games_with_name_df['movies_amount'].value_counts().head(10)

,count
movies_amount,
1,64375
2,13182
3,3977
4,1467
5,614
6,329
7,165
8,95
9,39


In [ ]:
games_df_description = describe_data(games_with_name_df)
games_df_description[games_df_description['null_amount'] > 0].sort_values(by='null_amount', ascending=False)

,non_null_amount,null_amount,unique_values,data_type
Tags,63506,20861,56707,object
Categories,79790,4577,5633,object
Publishers,80527,3840,43037,object
Developers,80802,3565,49516,object
Genres,80826,3541,2460,object


As colunas `Publishers` e `Developers` serão tratadas em seguida, sendo primeiramente verificados exemplos de dados de ambas.

In [ ]:
games_with_name_df.head(10)[['Publishers', 'Developers']]

,Publishers,Developers
17585,Valve,Valve
30583,Valve,Valve
8885,"KRAFTON, Inc.","KRAFTON, Inc."
46158,Valve,"Valve,Hidden Path Entertainment"
7030,Amazon Games,Amazon Games
33384,Coffee Stain Publishing,Iron Gate AB
39312,Starbreeze Publishing AB,OVERKILL - a Starbreeze Studio.
46416,Re-Logic,Re-Logic
47533,Ubisoft,Blue Mammoth Games
8256,Ubisoft,Ubisoft Montreal


É possível verificar que nem todo desenvolvedor de jogo é também o publicador daquele jogo, o que é esperado, já que principalmente em casos de grandes estúdios, ocorre a contratação de empresas menores para realizar o desenvolvimento de algum projeto.

Sendo assim, uma discriminação a ser feita é **verificar se o jogo foi ou não publicado por uma empresa considerada conhecida**. O julgamento de que empresas são consideradas conhecidas foi feito pelos autores, tomando como base o [Banco de Dados da Steam](https://steamdb.info/publishers/), que lista os publicadores de jogos mais populares da plataforma ordenados por avaliações positivas.

Com isso, as informações de desenvolvedores serão descartadas.

In [ ]:
MOST_POPULAR_PUBLISHERS = [
    'Valve',
    'Rockstar Games',
    'Electronic Arts',
    'Ubisoft',
    'Rockstar Games',
    'SEGA',
    'Bethesda',
    'CAPCOM',
    'Square Enix',
    'Bandai',
    'CD PROJEKT',
    'Paradox',
    'PlayStation',
    'Xbox',
    'Warner Bros',
    'KRAFTON',
    'Activison',
    'Re-Logic',
    'FromSoftware'
]

def is_publisher_known(publisher: str) -> bool:
  return any([popular_publisher in publisher for popular_publisher in MOST_POPULAR_PUBLISHERS])

games_with_name_df['Publishers'] = games_with_name_df['Publishers'].fillna('')
games_with_name_df['is_publisher_known'] = games_with_name_df['Publishers'].apply(is_publisher_known)

games_with_name_df.head(10)[['Name', 'Publishers', 'is_publisher_known']]

,Name,Publishers,is_publisher_known
17585,Dota 2,Valve,True
30583,Team Fortress 2,Valve,True
8885,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.",True
46158,Counter-Strike: Global Offensive,Valve,True
7030,New World,Amazon Games,False
33384,Valheim,Coffee Stain Publishing,False
39312,PAYDAY 2,Starbreeze Publishing AB,False
46416,Terraria,Re-Logic,True
47533,Brawlhalla,Ubisoft,True
8256,Tom Clancy's Rainbow Six® Siege,Ubisoft,True


In [ ]:
games_with_name_df = games_with_name_df.drop(columns=['Publishers'])
games_with_name_df = games_with_name_df.drop(columns=['Developers'])

games_df_description = describe_data(games_with_name_df)
games_df_description[games_df_description['null_amount'] > 0].sort_values(by='null_amount', ascending=False)

,non_null_amount,null_amount,unique_values,data_type
Tags,63506,20861,56707,object
Categories,79790,4577,5633,object
Genres,80826,3541,2460,object


As colunas `Tags`, `Genres` e `Categories` podem fornecer informações de texto valiosas, merecendo uma seção dedicada a exploração das mesmas.

A próxima seção tratará deste assunto.

### Transformando colunas categóricas com `one hot encode`
Colunas como `Tags`, `Genres` e `Categories` não são tratadas facilmente pelos algoritmos como valores categoricos, principalmente considerando as diferentes combinações de valores que podem ocorrer em uma única entrada.

Antes de aplicar a ideia de codificação dos valores em colunas binárias, será feita uma avaliação das distribuições de valores disponíveis.

In [ ]:
def get_freq_dist_from_column(column: pd.Series, column_name: str, separator: str = None) -> pd.DataFrame:
  column_name = column_name.replace(' ', '_').lower()
  freq_dist = {}

  column_values = column.dropna()
  if separator is not None:
    treated_column_values = []
    for value in column_values:
      treated_column_values += value.split(separator)
    column_values = treated_column_values

  for value in column_values:
    if value not in freq_dist:
      freq_dist[value] = 1
    else:
      freq_dist[value] += 1

  freq_dist = pd.DataFrame(list(freq_dist.items()), columns=[column_name + '_values', 'count'])
  freq_dist = freq_dist.sort_values(by='count', ascending=False)

  return freq_dist

tags_freq_dist = get_freq_dist_from_column(games_with_name_df['Tags'], 'Tags', ',')
tags_freq_dist

,tags_values,count
59,Indie,39949
60,Singleplayer,32866
7,Action,28532
71,Casual,27302
47,Adventure,26613
...,...,...
439,Birds,5
440,Hobby Sim,5
442,Coding,4
270,Extraction Shooter,3


In [ ]:
genres_freq_dist = get_freq_dist_from_column(games_with_name_df['Genres'], 'Genres', ',')
genres_freq_dist

,genres_values,count
6,Indie,56451
9,Casual,33929
0,Action,33483
3,Adventure,31161
8,Simulation,16044
2,Strategy,15469
5,RPG,14338
7,Early Access,10319
1,Free to Play,6549
14,Sports,3775


Analisando a quantidade, a diversidade e a qualidade das __tags__ em relação aos __genres__ optamos por manter a coluna __genres__ para a transformação dos rotulos em valores binários com One Hot Encoding.

In [ ]:
indie_share = genres_freq_dist[genres_freq_dist['genres_values'] == 'Indie']['count'].values[0] / games_with_name_df.shape[0]

print(f"{round(indie_share * 100, 3)}% dos jogos é do gênero Indie")

66.911% dos jogos é do gênero Indie


Mais de dois terços dos jogos da plataforma possuem o marcador `Indie`, o que pode levar a baixa variabilidade dos dados. Sendo assim, esse marcador será removido.

O mesmo deve ser feito no sentido contrário, onde marcadores com poucas ocorrências devem ser removidos. Serão considerados marcadores com menos de 1.000 ocorrências como de baixa relevância.

In [ ]:
genres_freq_dist = genres_freq_dist[genres_freq_dist['genres_values'] != 'Indie']

genres_freq_dist = genres_freq_dist[genres_freq_dist['count'] > 1_000]

genres_freq_dist

,genres_values,count
9,Casual,33929
0,Action,33483
3,Adventure,31161
8,Simulation,16044
2,Strategy,15469
5,RPG,14338
7,Early Access,10319
1,Free to Play,6549
14,Sports,3775
15,Racing,3077


Com essas alterações feitas, será gerado o `One hot encoding` desses dados.

In [ ]:
def verify_sufix_existence(column_value:str,sufix:str) -> bool:
  return sufix in column_value

def generate_one_hot_code(df: pd.DataFrame, column: str,dist: str):
  for sufix in dist:
    if sufix != 'NA':
      df[f'{column}_{sufix}'] = df[column].apply(lambda x: verify_sufix_existence(x,sufix))
  return df

games_with_name_df['Genres'].fillna('NA', inplace=True)

genres_freq_dist = genres_freq_dist['genres_values'].values


games_with_name_df = generate_one_hot_code(games_with_name_df,'Genres',genres_freq_dist)

O mesmo será feito para a coluna `Categories`.

In [ ]:
categories_freq_dist = get_freq_dist_from_column(games_with_name_df['Categories'], 'Categories', ',')

categories_freq_dist

,categories_values,count
22,Single-player,75749
8,Steam Achievements,36535
23,Steam Cloud,18688
18,Full controller support,15861
0,Multi-player,15460
10,Partial Controller Support,10522
2,Steam Trading Cards,9808
16,PvP,9772
1,Co-op,7742
17,Online PvP,6982


In [ ]:
single_player_share = categories_freq_dist[categories_freq_dist['categories_values'] == 'Single-player']['count'].values[0] / games_with_name_df.shape[0]

print(f"{round(single_player_share * 100, 3)}% dos jogos são Single-player")

89.785% dos jogos são Single-player


Assim como o que ocorreu com o marcador `Indie`, o marcador `Single-player` também será removido, já que quase 90% dos jogos possuem essa categoria. Da mesma forma, categorias com menos de 1.000 ocorrências serão removidas.

In [ ]:
categories_freq_dist = categories_freq_dist[categories_freq_dist['categories_values'] != 'Single-player']

categories_freq_dist = categories_freq_dist[categories_freq_dist['count'] > 1_000]

categories_freq_dist

,categories_values,count
8,Steam Achievements,36535
23,Steam Cloud,18688
18,Full controller support,15861
0,Multi-player,15460
10,Partial Controller Support,10522
2,Steam Trading Cards,9808
16,PvP,9772
1,Co-op,7742
17,Online PvP,6982
31,Steam Leaderboards,6771


In [ ]:
categories_freq_dist = categories_freq_dist['categories_values'].values

games_with_name_df['Categories'].fillna('NA', inplace=True)
games_with_name_df = generate_one_hot_code(games_with_name_df,'Categories',categories_freq_dist)

games_with_name_df.head(3)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,Supported languages,Full audio languages,...,Categories_Shared/Split Screen PvP,Categories_Stats,Categories_Shared/Split Screen Co-op,Categories_Cross-Platform Multiplayer,Categories_In-App Purchases,Categories_Remote Play on TV,Categories_Includes level editor,Categories_Steam Workshop,Categories_MMO,Categories_Captions available
17585,570,Dota 2,"Jul 9, 2013",100000000 - 200000000,558759,0,0.0,2,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['English', 'Korean', 'Simplified Chinese', 'V...",...,False,False,False,False,True,False,False,True,False,False
30583,440,Team Fortress 2,"Oct 10, 2007",50000000 - 100000000,107702,0,0.0,1,"['English', 'Danish', 'Dutch', 'Finnish', 'Fre...","['English', 'Ukrainian']",...,False,True,False,True,True,False,True,True,False,True
8885,578080,PUBG: BATTLEGROUNDS,"Dec 21, 2017",50000000 - 100000000,275374,0,0.0,0,"['English', 'Korean', 'Simplified Chinese', 'F...",[],...,False,True,False,False,False,False,False,False,False,False


Outra coluna que apresenta maior dificuldade de ser tratada é a que mostra a data de publicação do jogo. Sendo assim, ela também será tratada para aplicação da abordagem de codificação.

In [ ]:
def get_game_year(value: str) -> int:
  value_list = value.split(',')
  if len(value_list) == 2:
    return int(value_list[1])
  else:
    return 0

def get_game_season(value: str,seasons_dict: dict) -> str:
  for season in seasons_dict:
    for month in season:
      if month in value:
        return season
  return ''

def generate_game_seasons_column(df: pd.DataFrame):
  seasons_dict = {
      'winter': ['Dec', 'Jan', 'Feb'],
      'spring': ['Mar', 'Apr', 'May'],
      'summer': ['Jun', 'Jul', 'Aug'],
      'autumn': ['Sep', 'Oct', 'Nov']
  }
  df['year'] = df['Release date'].apply(get_game_year)
  df['season'] = df['Release date'].apply(lambda x: get_game_season(x,seasons_dict))
  return df

games_with_name_df['Release date'].fillna('',inplace=True)
games_with_name_df = generate_game_seasons_column(games_with_name_df)

# Fazendo o one hot encoding dos valores das estações do ano
dumies = pd.get_dummies(games_with_name_df['season'])
games_with_name_df = pd.concat([games_with_name_df, dumies], axis=1)
games_with_name_df.drop('season', axis=1, inplace=True)

# Exibindo os resultados
games_with_name_df.head(3)[['Name','Release date','year', 'winter', 'spring', 'autumn', 'summer']]

,Name,Release date,year,winter,spring,autumn,summer
17585,Dota 2,"Jul 9, 2013",2013,False,False,False,True
30583,Team Fortress 2,"Oct 10, 2007",2007,True,False,False,False
8885,PUBG: BATTLEGROUNDS,"Dec 21, 2017",2017,True,False,False,False


As colunas `DLC count` e `Peak CCU` serão mantidas como valores numéricos, já que podem trazer informações úteis para a análise.

Por fim, a coluna `Estimated owners` também será codificada na mesma abordagem utilizando os intervalos originais dos dados.

In [ ]:
games_with_name_df = generate_one_hot_code(games_with_name_df, 'Estimated owners', games_with_name_df['Estimated owners'].unique())

games_with_name_df.head(3)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,Supported languages,Full audio languages,...,Estimated owners_5000000 - 10000000,Estimated owners_2000000 - 5000000,Estimated owners_1000000 - 2000000,Estimated owners_500000 - 1000000,Estimated owners_200000 - 500000,Estimated owners_100000 - 200000,Estimated owners_50000 - 100000,Estimated owners_20000 - 50000,Estimated owners_0 - 20000,Estimated owners_0 - 0
17585,570,Dota 2,"Jul 9, 2013",100000000 - 200000000,558759,0,0.0,2,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['English', 'Korean', 'Simplified Chinese', 'V...",...,False,False,False,False,False,False,False,False,True,False
30583,440,Team Fortress 2,"Oct 10, 2007",50000000 - 100000000,107702,0,0.0,1,"['English', 'Danish', 'Dutch', 'Finnish', 'Fre...","['English', 'Ukrainian']",...,False,False,False,False,False,False,False,False,False,False
8885,578080,PUBG: BATTLEGROUNDS,"Dec 21, 2017",50000000 - 100000000,275374,0,0.0,0,"['English', 'Korean', 'Simplified Chinese', 'F...",[],...,False,False,False,False,False,False,False,False,False,False


## Eliminando colunas desnecessárias
Após a etapa de tratamento dos dados, é necessário limpar a base das colunas desnecessárias para a execução do algoritmo.

In [ ]:
describe_data(games_with_name_df).head(30)

,non_null_amount,null_amount,unique_values,data_type
AppID,84367,0,84367,int64
Name,84367,0,84367,object
Release date,84367,0,4469,object
Estimated owners,84367,0,14,object
Peak CCU,84367,0,1439,int64
Required age,84367,0,19,int64
Price,84367,0,584,float64
DLC count,84367,0,95,int64
Supported languages,84367,0,11267,object
Full audio languages,84367,0,2230,object


As colunas `AppId`, `Name`, `Release date`, `Supported languages`, `Full audio languages`, `Header image`, `Windows`, `Mac`, `Linux` e `Tags` não serão utilizadas para a análise de subgrupos, portanto, serão removidas.

As colunas `Average playtime forever`, `Median playtime forever`, `Average playtime last two weeks` e `Median playtime last two weeks`, por terem poucas informações que ajudem a identificar a forma de cálculo desses valores, serão removidas.

In [ ]:
columns_to_drop = [
    'AppID',
    'Name',
    'Release date',
    'Supported languages',
    'Full audio languages',
    'Header image',
    'Windows',
    'Mac',
    'Linux',
    'Tags',
    'Average playtime forever',
    'Average playtime two weeks',
    'Median playtime forever',
    'Median playtime two weeks',
]

cleaned_df = games_with_name_df.drop(columns=columns_to_drop)

described_df = describe_data(cleaned_df)
described_df.head(30)

,non_null_amount,null_amount,unique_values,data_type
Estimated owners,84367,0,14,object
Peak CCU,84367,0,1439,int64
Required age,84367,0,19,int64
Price,84367,0,584,float64
DLC count,84367,0,95,int64
Metacritic score,84367,0,73,int64
User score,84367,0,33,int64
Positive,84367,0,4472,int64
Negative,84367,0,2250,int64
Achievements,84367,0,429,int64


A ideia é transformar todas as colunas em valores binários, para que possam ser utilizadas de forma mais eficiente na análise de subgrupos.

In [ ]:
described_df[described_df['data_type'] != 'bool']

,non_null_amount,null_amount,unique_values,data_type
Estimated owners,84367,0,14,object
Peak CCU,84367,0,1439,int64
Required age,84367,0,19,int64
Price,84367,0,584,float64
DLC count,84367,0,95,int64
Metacritic score,84367,0,73,int64
User score,84367,0,33,int64
Positive,84367,0,4472,int64
Negative,84367,0,2250,int64
Achievements,84367,0,429,int64


Após análises preliminares, executando algoritmos de descoberta de subgrupos nos dados, as colunas `Required age`, `Price`, `Achievements`, `Recommendations`, `screenshots_amount`, `movies_amount`, `Estimated owners` e `Estimated owners mean` serão removidas por trazerem associações muito genéricas para análise.

As colunas `Categories` e `Genres`, por já terem sido processadas, também serão removidas.

In [ ]:
columns_to_drop = [
    'Required age',
    'Price',
    'Achievements',
    'Recommendations',
    'screenshots_amount',
    'movies_amount',
    'Estimated owners',
    'Estimated owners mean',
    'Categories',
    'Genres'
]

cleaned_df = cleaned_df.drop(columns=columns_to_drop)

described_df = describe_data(cleaned_df)
described_df[described_df['data_type'] != 'bool']

,non_null_amount,null_amount,unique_values,data_type
Peak CCU,84367,0,1439,int64
DLC count,84367,0,95,int64
Metacritic score,84367,0,73,int64
User score,84367,0,33,int64
Positive,84367,0,4472,int64
Negative,84367,0,2250,int64
year,84367,0,30,int64


## Execução do Beam Search
As células a seguir fazem a preparação dos dados para a execução do algoritmo Beam Search.

In [ ]:
!pip install pysubgroup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pysubgroup as ps

df_test = cleaned_df.copy()
df_test["target"] = (df_test["Positive"]>df_test["Negative"]*1.2) & (df_test["Positive"]> df_test["Positive"].mean())

In [ ]:
ignore_numeric_columns = [
    "Positive",
    "Negative",
    "target"
]

ignore_nominal_columns = [
    "target"
]

In [ ]:
target = ps.BinaryTarget('target', True)
searc_space = ps.create_nominal_selectors(df_test, ignore=ignore_nominal_columns)
searc_space += ps.create_numeric_selectors(df_test, ignore=ignore_numeric_columns)

task = ps.SubgroupDiscoveryTask(df_test, target, searc_space, result_set_size=20, depth=100000, qf=ps.WRAccQF())

result = ps.algorithms.BeamSearch().execute(task)

result_df = result.to_dataframe()

result_df.to_csv('result.csv',index=False)

## Preparando os arquivos para execução do MCTS

### Gerando o qualities.csv

In [ ]:
avg_positive = cleaned_df['Positive'].mean()
def gen_qualities(df_line ):
  return 1 if (df_line["Positive"]>df_line["Negative"]*1.2) & (df_line["Positive"]> avg_positive) else 0
df_quality = cleaned_df.apply(gen_qualities, axis=1)
df_quality.to_csv("qualities.csv", index=False)

### Removendo colunas que não serão consideradas

In [ ]:
df = cleaned_df.copy()
describe_data(df)
columns_to_drop = [
    "Positive",
    "Negative"
]

df = df.drop(columns=columns_to_drop)
boolean_columns =  df.columns[df.dtypes == bool]
for col in boolean_columns:
    df[col] = df[col].map(lambda x: 1 if x else 0)

df.to_csv("properties.csv", index=False)

In [ ]:
describe_data(df)

,non_null_amount,null_amount,unique_values,data_type
Peak CCU,84367,0,1439,int64
DLC count,84367,0,95,int64
Metacritic score,84367,0,73,int64
User score,84367,0,33,int64
has_review,84367,0,2,int64
...,...,...,...,...
Estimated owners_100000 - 200000,84367,0,2,int64
Estimated owners_50000 - 100000,84367,0,2,int64
Estimated owners_20000 - 50000,84367,0,2,int64
Estimated owners_0 - 20000,84367,0,2,int64


### Exporta dataset arff

In [ ]:
!pip install arff
import arff

arff.dump('Games.arff'
      , df.values
      , relation='Games'
      , names=df.columns)

## Verificação de exemplos dos subgrupos
Foram selecionados os subgrupos mais interessantes, segundo avaliação dos autores, para visualizar exemplos de jogos que se enquadram neles.

### Jogos de sucesso contra a crítica

In [ ]:
games_with_name_df[
    (games_with_name_df['Metacritic score'].between(51, 74)) &
    (games_with_name_df['Categories_Steam Leaderboards'] == 1.0) &
    (games_with_name_df['Estimated owners_2000000 - 5000000'] == 1.0)
][['Name', 'Genres', 'Categories']]

,Name,Genres,Categories
31616,Call of Duty®: Black Ops II,Action,"Single-player,Multi-player,Co-op,Steam Achieve..."
20378,Resident Evil 6,"Action,Adventure","Single-player,Multi-player,Co-op,Shared/Split ..."
77287,F1 2015,"Racing,Sports","Single-player,Multi-player,Cross-Platform Mult..."
15118,Call of Duty®: Black Ops III,"Action,Adventure","Single-player,Multi-player,Co-op,Shared/Split ..."
17577,Magicka,"Action,RPG","Single-player,Multi-player,Co-op,Shared/Split ..."
53320,Goat Simulator,"Casual,Indie,Simulation","Single-player,Multi-player,Shared/Split Screen..."
52270,Friday the 13th: The Game,Action,"Multi-player,Steam Achievements,Full controlle..."
37438,"Warhammer 40,000: Space Marine - Anniversary E...",Action,"Single-player,Multi-player,Steam Achievements,..."
30239,Batman™: Arkham Origins,"Action,Adventure","Single-player,Steam Achievements,Full controll..."
21488,Serious Sam 3: BFE,"Action,Indie","Single-player,Multi-player,PvP,Online PvP,Shar..."


### Jogos de dedicação exclusiva

In [ ]:
games_with_name_df[
    (games_with_name_df['Peak CCU'].between(440, 2650)) &
    (games_with_name_df['is_publisher_known'] == 1.0) &
    (games_with_name_df['Categories_Multi-player'] == 1.0) &
    (games_with_name_df['Categories_Steam Workshop'] == 1.0) &
    (games_with_name_df['Estimated owners_2000000 - 5000000'] == 0.0)
][['Name', 'Genres', 'Categories', 'Estimated owners']]

,Name,Genres,Categories,Estimated owners
52769,Age of Wonders III,"RPG,Strategy","Single-player,Multi-player,Co-op,Shared/Split ...",1000000 - 2000000
26259,Command & Conquer™ Remastered Collection,Strategy,"Single-player,Multi-player,PvP,Online PvP,Stea...",1000000 - 2000000
49719,F1® 2020,"Racing,Simulation,Sports","Single-player,Multi-player,PvP,Online PvP,Stea...",1000000 - 2000000
46739,Football Manager 2019,"Simulation,Sports","Single-player,Multi-player,Steam Achievements,...",1000000 - 2000000
5431,Age of Mythology: Extended Edition,"Simulation,Strategy","Single-player,Multi-player,Co-op,Steam Achieve...",1000000 - 2000000
34868,Company of Heroes,"Action,Strategy","Single-player,Multi-player,Steam Trading Cards...",1000000 - 2000000
44739,Imperator: Rome,"Simulation,Strategy","Single-player,Multi-player,PvP,Online PvP,Stea...",500000 - 1000000
48368,ENDLESS™ Space 2,Strategy,"Single-player,Multi-player,PvP,Online PvP,Cros...",200000 - 500000
33472,Age of Wonders: Planetfall,Strategy,"Single-player,Multi-player,PvP,Online PvP,Stea...",200000 - 500000
28945,Total War: ROME REMASTERED,Strategy,"Single-player,Multi-player,Cross-Platform Mult...",200000 - 500000
